In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [2]:
from formula.cnf import CNF

# Read in 100 randomly generated CNF(12, 8, r_8) instances
num_instances = 100
n = 12
k = 8

instances = [] 

for i in range(num_instances):

	# Use correct directory
	parent_dir = os.path.dirname(os.getcwd())
	dir = f"{parent_dir}/benchmark/instances/n_{n}"
	cnf_filename = f"{dir}/f_n{n}_k{k}_{i}.cnf"

	instances.append(CNF.from_file(cnf_filename))

In [3]:
import torch
f = instances[3]
torch.sum(f.naive_sats)

tensor(2.)

In [6]:
from k_sat.pytorch_solver.pytorch_solver import PytorchSolver

ps = PytorchSolver(layers=3)
ps.sat(f, timeout=200)

Initialising network
Finding optimal params
Success probability: 0.0005169192445464432
Success probability: 0.025851774960756302
Success probability: 0.026220351457595825
Success probability: 0.026222314685583115
Success probability: 0.0262223519384861


('000101010101', 4)

In [5]:
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit
from k_sat.pytorch_solver.pytorch_optimiser import PytorchOptimiser

ps = [1] #2, 4, 8, 30, 60]
optimal_params = {}

for p in ps:
	circuit = PytorchCircuit(num_vars=n, layers=p)
	optimiser = PytorchOptimiser(circuit)
	optimal_params[p] = optimiser.find_optimal_params(instances)

KeyboardInterrupt: 

In [ ]:
optimal_params

{1: array([0.57389736, 0.44572303])}

Step 2: Evaluate on satisfiable CNF instances

In [ ]:
from k_sat.walkSATlm.walkSATlm import WalkSATlm
from k_sat.quantum_solver.qaoa_evaluator import QAOAEvaluator
from statistics import median

# QAOA
encoder = PauliEncoder()
evaluator = QAOAEvaluator()

# TODO: evaluate n_e for [12...20], num_instances_e = 10000
k_e = k
n_e = [12]
num_instances_e = 5

# Dictionary results
p_succ = {n : {} for n in n_e}
running_times = {n : {} for n in n_e}

for n in n_e:
	print(f'evaluating n = {n}')
	# Generate satisfiable instances
	instances_e = RandomKSAT.from_poisson(n, k_e, satisfiable=True, instances = num_instances_e)
	for (p, params) in optimal_params.items():
		print(f'evaluating p = {p}')
		p_succ_p = 0
		running_times_p = []

		# Evaluate success probability and running time for QAOA
		for formula in instances_e.formulas:
			circuit = encoder.encode_formula(formula, p)
			p_succ_p += evaluator.success_probability(circuit, formula, params)
			ass, running_time_f = evaluator.running_time(circuit, formula, params, timeout=100) 
			if ass == "-1":
				print("QAOA TIMEOUT")
			running_times_p.append(running_time_f)

		# Store results
		p_succ[n][p] = p_succ_p / num_instances_e
		print(running_times_p)
		running_times[n][p] = median(running_times_p)


evaluating n = 12
Walksatlm took 100001000 steps
Walksatlm took 3 steps
Walksatlm took 2 steps
Walksatlm took 100001000 steps
Walksatlm took 8 steps
evaluating p = 1
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
QAOA TIMEOUT
[1, 1, 1, 1, 1]


In [ ]:
p_succ

{12: {1: 0.017068698906207914}}

In [ ]:
running_times

{12: {1: 1}}